<a href="https://colab.research.google.com/github/ANUPRIYAR/ML_Practise/blob/main/CNN_kerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=31873b730e76959d399e942b63e13d299f476dbd32ff46c77d23dc981eeb99bf
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=9394920080277c7520c578f82323695d0139df220a60ac3c3fc46788e90afafe
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
tf.__version__

'2.4.0'

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
train_images = train_images/255
test_images = test_images/255

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [17]:
def build_model(hp):
  model = keras.Sequential([
               keras.layers.Conv2D(
                   filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                   kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
                   activation='relu'
               ),
                 keras.layers.Conv2D(
                   filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
                   kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
                   activation='relu'
               ),   
               keras.layers.Flatten(),
               keras.layers.Dense(
                   units=hp.Int('dense_1_units',min_value=32, max_value=128, step=16),
                   activation='relu'
               ),
               keras.layers.Dense(10,activation='softmax')                                    
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model



In [15]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name='Fashion_mnist')

In [19]:
tuner_search.search(train_images,train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 28s]
val_accuracy: 0.1003333330154419

Best val_accuracy So Far: 0.8351666927337646
Total elapsed time: 00h 02m 23s
INFO:tensorflow:Oracle triggered exit


In [20]:
model = tuner_search.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 112)       268912    
_________________________________________________________________
flatten (Flatten)            (None, 54208)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                4336720   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 4,607,402
Trainable params: 4,607,402
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(train_images, train_labels, validation_split=0.1, epochs=10, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4331 - accuracy: 0.8444 - val_loss: 0.4045 - val_accuracy: 0.8495
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4027 - accuracy: 0.8551 - val_loss: 0.3866 - val_accuracy: 0.8578
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3781 - accuracy: 0.8630 - val_loss: 0.3941 - val_accuracy: 0.8513
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3607 - accuracy: 0.8713 - val_loss: 0.3555 - val_accuracy: 0.8727
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3447 - accuracy: 0.8745 - val_loss: 0.3614 - val_accuracy: 0.8705
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3297 - accuracy: 0.8790 - val_loss: 0.3342 - val_accuracy: 0.8780
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3147 - accuracy: 0.8847 - val_loss: 0.3489 - val_a